In [1]:
#from google.colab import files, drive
#drive.mount('/content/drive')

In [2]:
#!pip install -r ./drive/MyDrive/CMC/requirements.txt

In [3]:
'''
from __future__ import absolute_import

import sys
import os

sys.path.append('./drive/My Drive/CMC/')
'''

"\nfrom __future__ import absolute_import\n\nimport sys\nimport os\n\nsys.path.append('./drive/My Drive/CMC/')\n"

In [4]:
#!wandb login

In [5]:
from __future__ import absolute_import

import sys
import os

import shutil

try:
    from dotenv import find_dotenv, load_dotenv
except:
    pass

import argparse

try:
    sys.path.append(os.path.join(os.path.dirname(__file__), './drive/My Drive/CMC/'))
except:
    sys.path.append(os.path.join(os.getcwd(), './drive/My Drive/CMC/'))
    
try:
    sys.path.append(os.path.join(os.path.dirname(__file__), '../'))
except:
    sys.path.append(os.path.join(os.getcwd(), '../'))
    
import pandas as pd
import numpy as np

import pickle
from collections import Counter
from tqdm import tqdm

import tensorflow as tf
import tensorflow.keras.backend as K
import tensorflow_addons as tfa

try:
    import wandb
    load_dotenv(find_dotenv())
    wandb.login(key=os.environ['WANDB_API_KEY'])
    from wandb.keras import WandbCallback
    _has_wandb = True
except:
    _has_wandb = False

import tokenizers
from transformers import TFAutoModel, AutoTokenizer, AutoConfig, BertTokenizer

from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer

from src import data, models

pd.options.display.max_colwidth = -1

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publically.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: WARNING Calling wandb.login() without arguments from jupyter should prompt you for an api key.
wandb: Appending key for api.wandb.ai to your netrc file: /Users/victor/.netrc
I0110 13:09:06.326184 4591582720 file_utils.py:41] PyTorch version 1.4.0 available.
I0110 13:09:06.326936 4591582720 file_utils.py:57] TensorFlow version 2.2.0-rc3 available.
/Users/victor/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:54: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.


In [6]:
print (_has_wandb)

True


In [7]:
parser = argparse.ArgumentParser(prog='Trainer',conflict_handler='resolve')

#parser.add_argument('--train_data', type=str, default='./drive/My Drive/CMC/data/IIITH_Codemixed.txt', required=False,
#                    help='train data')
parser.add_argument('--train_data', type=str, default='../data/raw/Hate-speech-dataset/hate_speech.tsv', required=False,
                    help='train data')
parser.add_argument('--val_data', type=str, default=None, required=False,
                    help='validation data')
parser.add_argument('--test_data', type=str, default=None, required=False,
                    help='test data')

parser.add_argument('--transformer_model_pretrained_path', type=str, default='roberta-base', required=False,
                    help='transformer model pretrained path or huggingface model name')
parser.add_argument('--transformer_config_path', type=str, default='roberta-base', required=False,
                    help='transformer config file path or huggingface model name')
parser.add_argument('--transformer_tokenizer_path', type=str, default='roberta-base', required=False,
                    help='transformer tokenizer file path or huggingface model name')

parser.add_argument('--max_text_len', type=int, default=50, required=False,
                    help='maximum length of text')
parser.add_argument('--max_char_len', type=int, default=200, required=False,
                    help='maximum length of text')
parser.add_argument('--max_word_char_len', type=int, default=20, required=False,
                    help='maximum length of text')

parser.add_argument('--emb_dim', type=int, default=128, required=False,
                    help='maximum length of text')
parser.add_argument('--n_layers', type=int, default=2, required=False,
                    help='maximum length of text')
parser.add_argument('--n_units', type=int, default=128, required=False,
                    help='maximum length of text')

parser.add_argument('--epochs', type=int, default=500, required=False,
                    help='number of epochs')
parser.add_argument('--lr', type=float, default=.001, required=False,
                    help='learning rate')
parser.add_argument('--early_stopping_rounds', type=int, default=50, required=False,
                    help='number of epochs for early stopping')
parser.add_argument('--lr_schedule_round', type=int, default=30, required=False,
                    help='number of epochs for learning rate scheduling')

parser.add_argument('--train_batch_size', type=int, default=32, required=False,
                    help='train batch size')
parser.add_argument('--eval_batch_size', type=int, default=16, required=False,
                    help='eval batch size')

#parser.add_argument('--model_save_path', type=str, default='./drive/My Drive/CMC/models/model_hindi_sentiment/', required=False,
#                    help='seed')

parser.add_argument('--model_save_path', type=str, default='../models/hate_detection/', required=False,
                    help='seed')

parser.add_argument('--wandb_logging', type=bool, default=True, required=False,
                    help='wandb logging needed')

parser.add_argument('--seed', type=int, default=42, required=False,
                    help='seed')


args, _ = parser.parse_known_args()

In [8]:
tf.random.set_seed(args.seed)
np.random.seed(args.seed)

In [9]:
df = pd.read_csv(args.train_data, sep='\t',header=None,usecols=[0,1])
df.columns = ['text','category']
df = df.dropna()
df = df[df.text != '']

kf = KFold(n_splits=5, shuffle=True, random_state=args.seed)
for train_index, test_index in kf.split(df.text):
    break

train_df = df.iloc[train_index]
kf2 = KFold(n_splits=2, shuffle=True, random_state=args.seed)
for val_index, test_index in kf2.split(df.iloc[test_index].text):
    break

val_df = df.iloc[val_index]
test_df = df.iloc[test_index]

I0110 13:09:08.364080 4591582720 utils.py:141] NumExpr defaulting to 4 threads.


In [10]:
print (train_df.shape, val_df.shape, test_df.shape)

(3662, 2) (458, 2) (458, 2)


In [11]:
train_df.head(5)

,text,category
0,"Knowing ki Vikas kitna samjhata hai Priyanka aur Itch Guard Luv ko, usne bola tha Ben wali baat me ab Sallu ne bhi agree kiya!",no
1,I am Muhajir .. Aur mere lye sab se Pehly Pakistan he .. agr 10 lakh Altaf Jese leaders bh is zameen ki behurmati kren un sbko sar e aam phansi Deni chahye .. Proud to be a #Muhajir and #Pakistani,no
2,"Doctor sab sahi me ke PhD (in hate politics) wale. Bhai padhe likhe ho fir kyu ye sab baate karte ho. Tum bas bowling khelo, aur maje lo. pic.twitter.com/fk1qUbQstw",no
3,"Poore Desh me Patel OBC me aate Hain sirf gujrat Ko chor kar may be, ye manuwadiyon bramanwadi kabhi aapko aarackchan nahi denge ye to jis OBC Ko Mila hai usse bhi nafrat karte hain ye khoon aur chamdi ka frak karne waale bharmhanwadi kisi ke sage nahi hain",no
4,"Sarkar banne ke bad Hindu hit me ek bhi faisla Jo bjp ke dwara liya gaya ho,bjp ko gay,gobar,mandir,masjid aur nafrat faila kar vot chahiye",yes


In [12]:
train_df.text = train_df.text.apply(lambda x: data.preprocessing.clean_tweets(x))
val_df.text = val_df.text.apply(lambda x: data.preprocessing.clean_tweets(x))
test_df.text = test_df.text.apply(lambda x: data.preprocessing.clean_tweets(x))

/Users/victor/.local/lib/python3.7/site-packages/pandas/core/generic.py:5165: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [13]:
train_df = train_df[train_df.text != '']
val_df = val_df[val_df.text != '']
test_df = test_df[test_df.text != '']


In [14]:
train_df.text.apply(lambda x: len(x)).describe()

count    3661.000000
mean     102.369571 
std      53.175251  
min      6.000000   
25%      63.000000  
50%      99.000000  
75%      130.000000 
max      313.000000 
Name: text, dtype: float64

In [15]:
train_df.text.apply(lambda x: len(x.split())).describe()

count    3661.000000
mean     19.663207  
std      10.279373  
min      1.000000   
25%      12.000000  
50%      19.000000  
75%      25.000000  
max      64.000000  
Name: text, dtype: float64

In [16]:
model_save_dir = args.model_save_path

try:
    os.makedirs(model_save_dir)
except OSError:
    pass

In [17]:
train_df.category.value_counts()

no     2316
yes    1342
on     2   
n      1   
Name: category, dtype: int64

In [18]:
train_df = train_df[train_df.category.str.contains('yes|no')]
val_df = val_df[val_df.category.str.contains('yes|no')]
test_df = test_df[test_df.category.str.contains('yes|no')]


In [19]:
model_save_dir

'../models/hate_detection/'

In [20]:
train_df.category, label2idx = data.data_utils.convert_categorical_label_to_int(train_df.category.values, \
                                                         save_path=os.path.join(model_save_dir,'label2idx.pkl'))

val_df.category, _ = data.data_utils.convert_categorical_label_to_int(val_df.category.values, \
                                                         save_path=os.path.join(model_save_dir,'label2idx.pkl'))

test_df.category, _ = data.data_utils.convert_categorical_label_to_int(test_df.category.values, \
                                                         save_path=os.path.join(model_save_dir,'label2idx.pkl'))

In [21]:
train_df.head(5)

,text,category
0,"knowing ki vikas kitna samjhata hai priyanka aur itch guard luv ko, usne bola tha ben wali baat me ab sallu ne bhi agree kiya!",0
1,i am muhajir .. aur mere lye sab se pehly pakistan he .. agr lakh altaf jese leaders bh is zameen ki behurmati kren un sbko sar e aam phansi deni chahye .. proud to be a and,0
2,"doctor sab sahi me ke phd (in hate politics) wale. bhai padhe likhe ho fir kyu ye sab baate karte ho. tum bas bowling khelo, aur maje lo. pic.twitter.com/fkqubqstw",0
3,"poore desh me patel obc me aate hain sirf gujrat ko chor kar may be, ye manuwadiyon bramanwadi kabhi aapko aarackchan nahi denge ye to jis obc ko mila hai usse bhi nafrat karte hain ye khoon aur chamdi ka frak karne waale bharmhanwadi kisi ke sage nahi hain",0
4,"sarkar banne ke bad hindu hit me ek bhi faisla jo bjp ke dwara liya gaya ho,bjp ko gay,gobar,mandir,masjid aur nafrat faila kar vot chahiye",1


In [22]:
print (label2idx)

{'no': 0, 'yes': 1}


In [23]:
idx2label = {i:w for (w,i) in label2idx.items()}

### Learn tokenizer

In [24]:
data.custom_tokenizers.custom_wp_tokenizer(train_df.text.values, args.model_save_path, args.model_save_path)
tokenizer = BertTokenizer.from_pretrained(args.model_save_path)

I0110 13:09:09.044483 4591582720 tokenization_utils.py:420] Model name '../models/hate_detection/' not found in model shortcut name list (bert-base-uncased, bert-large-uncased, bert-base-cased, bert-large-cased, bert-base-multilingual-uncased, bert-base-multilingual-cased, bert-base-chinese, bert-base-german-cased, bert-large-uncased-whole-word-masking, bert-large-cased-whole-word-masking, bert-large-uncased-whole-word-masking-finetuned-squad, bert-large-cased-whole-word-masking-finetuned-squad, bert-base-cased-finetuned-mrpc, bert-base-german-dbmdz-cased, bert-base-german-dbmdz-uncased, bert-base-finnish-cased-v1, bert-base-finnish-uncased-v1, bert-base-dutch-cased). Assuming '../models/hate_detection/' is a path, a model identifier, or url to a directory containing tokenizer files.
I0110 13:09:09.046682 4591582720 tokenization_utils.py:449] Didn't find file ../models/hate_detection/added_tokens.json. We won't load it.
I0110 13:09:09.047651 4591582720 tokenization_utils.py:449] Didn't

In [25]:
word_tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=50000, split=' ',oov_token=1)
char_tokenizer = tf.keras.preprocessing.text.Tokenizer(char_level=True, split='',oov_token=1)

word_tokenizer.fit_on_texts(train_df.text.values)
char_tokenizer.fit_on_texts(train_df.text.values)

In [26]:
transformer_train_inputs, _, _ = data.data_utils.compute_transformer_input_arrays(train_df, 'text', tokenizer, args.max_char_len)

word_train_inputs = word_tokenizer.texts_to_sequences(train_df.text.values)
word_train_inputs = tf.keras.preprocessing.sequence.pad_sequences(word_train_inputs, maxlen=args.max_text_len)

subword_train_inputs = np.asarray([data.data_utils.subword_tokenization(text, char_tokenizer, args.max_text_len, args.max_word_char_len) \
                        for text in tqdm(train_df.text.values)])

char_train_inputs = char_tokenizer.texts_to_sequences(train_df.text.values)
char_train_inputs = tf.keras.preprocessing.sequence.pad_sequences(char_train_inputs, maxlen=args.max_char_len)

train_outputs = data.data_utils.compute_output_arrays(train_df, 'category')

transformer_val_inputs, _, _ = data.data_utils.compute_transformer_input_arrays(val_df, 'text', tokenizer, args.max_char_len)

word_val_inputs = word_tokenizer.texts_to_sequences(val_df.text.values)
word_val_inputs = tf.keras.preprocessing.sequence.pad_sequences(word_val_inputs, maxlen=args.max_text_len)

subword_val_inputs = np.asarray([data.data_utils.subword_tokenization(text, char_tokenizer, args.max_text_len, args.max_word_char_len) \
                        for text in tqdm(val_df.text.values)])

char_val_inputs = char_tokenizer.texts_to_sequences(val_df.text.values)
char_val_inputs = tf.keras.preprocessing.sequence.pad_sequences(char_val_inputs, maxlen=args.max_char_len)

val_outputs = data.data_utils.compute_output_arrays(val_df, 'category')

transformer_test_inputs, _, _ = data.data_utils.compute_transformer_input_arrays(test_df, 'text', tokenizer, args.max_char_len)

word_test_inputs = word_tokenizer.texts_to_sequences(test_df.text.values)
word_test_inputs = tf.keras.preprocessing.sequence.pad_sequences(word_test_inputs, maxlen=args.max_text_len)

subword_test_inputs = np.asarray([data.data_utils.subword_tokenization(text, char_tokenizer, args.max_text_len, args.max_word_char_len) \
                        for text in tqdm(test_df.text.values)])

char_test_inputs = char_tokenizer.texts_to_sequences(test_df.text.values)
char_test_inputs = tf.keras.preprocessing.sequence.pad_sequences(char_test_inputs, maxlen=args.max_char_len)

test_outputs = data.data_utils.compute_output_arrays(test_df, 'category')

train_outputs = tf.keras.utils.to_categorical(train_outputs, \
                                                    num_classes=train_df.category.nunique())
val_outputs = tf.keras.utils.to_categorical(val_outputs, \
                                                    num_classes=train_df.category.nunique())
test_outputs = tf.keras.utils.to_categorical(test_outputs, \
                                                    num_classes=train_df.category.nunique())

tfidf1 = TfidfVectorizer(stop_words='english',ngram_range=(1,3), max_df=.6,min_df=2)
tfidf2 = TfidfVectorizer(analyzer='char_wb',ngram_range=(1,3), max_df=.6,min_df=2)

tfidf1.fit(train_df.text)
tfidf2.fit(train_df.text)

train_tfidf = np.hstack([tfidf1.transform(train_df.text).toarray(),tfidf2.transform(train_df.text).toarray()])
val_tfidf = np.hstack([tfidf1.transform(val_df.text).toarray(),tfidf2.transform(val_df.text).toarray()])
test_tfidf = np.hstack([tfidf1.transform(test_df.text).toarray(),tfidf2.transform(test_df.text).toarray()])


print (transformer_train_inputs.shape, subword_train_inputs.shape, word_train_inputs.shape, char_train_inputs.shape, \
       train_tfidf.shape, train_outputs.shape)
print (transformer_val_inputs.shape, subword_val_inputs.shape, word_val_inputs.shape, char_val_inputs.shape, \
       val_tfidf.shape, val_outputs.shape)
print (transformer_test_inputs.shape, subword_test_inputs.shape, word_test_inputs.shape, char_test_inputs.shape, \
       test_tfidf.shape, test_outputs.shape)


100%|██████████| 458/458 [00:00<00:00, 1892.70it/s]


(3658, 200) (3658, 50, 20) (3658, 50) (3658, 200) (3658, 18201) (3658, 2)
(457, 200) (457, 50, 20) (457, 50) (457, 200) (457, 18201) (457, 2)
(458, 200) (458, 50, 20) (458, 50) (458, 200) (458, 18201) (458, 2)


### Modeling

In [27]:
n_words = len(word_tokenizer.word_index)+1
n_chars = len(char_tokenizer.word_index)+1
n_subwords = tokenizer.vocab_size
tfidf_shape = train_tfidf.shape[1]
n_out = train_df.category.nunique()

In [28]:
'''
mha = MultiHeadSelfAttention(128,8)
x = tf.constant(np.random.random((16,20,128)))
out = mha(x)
out

mha = OuterProductMHSA(128)
x = tf.constant(np.random.random((16,20,128)))
out = mha(x)
out
'''

'\nmha = MultiHeadSelfAttention(128,8)\nx = tf.constant(np.random.random((16,20,128)))\nout = mha(x)\nout\n\nmha = OuterProductMHSA(128)\nx = tf.constant(np.random.random((16,20,128)))\nout = mha(x)\nout\n'

In [29]:
from src.models.models import *

all_models = {CS_ELMO_without_words.__name__: CS_ELMO_without_words, Transformer.__name__: Transformer, HAN.__name__: HAN}


In [30]:
_has_wandb = False

In [ ]:
if os.path.exists(os.path.join(args.model_save_path,'results.csv')):
  results = pd.read_csv(os.path.join(args.model_save_path,'results.csv'))
  index = results.shape[0]
  print (results)
else:
  results = pd.DataFrame(columns=['config','weighted_f1','macro_f1'])
  index = 0

for model_name, model_ in all_models.items():
    
    for loss in ['ce','focal']:
        
        for use_features in [False]:
            
            if use_features == False:
                model = model_(word_vocab_size=n_words,char_vocab_size=n_chars,wpe_vocab_size=n_subwords, n_out=n_out,max_word_char_len=args.max_word_char_len,\
                                             max_text_len=args.max_text_len, max_char_len=args.max_char_len,\
                                             n_layers=args.n_layers, n_units=args.n_units, emb_dim=args.emb_dim)
            else:
                model = model_(word_vocab_size=n_words,char_vocab_size=n_chars,wpe_vocab_size=n_subwords,n_out=n_out,vectorizer_shape=tfidf_shape, max_word_char_len=args.max_word_char_len,\
                                             max_text_len=args.max_text_len, max_char_len=args.max_char_len,\
                                             n_layers=args.n_layers, n_units=args.n_units, emb_dim=args.emb_dim)
            
            if use_features == True:
                print ("Running {} with features for {} loss".format(model_name, loss))
            else:
                print ("Running {} without features for {} loss".format(model_name, loss))

            print (model.summary())

            if loss == 'focal':
                model.compile(loss=models.utils.categorical_focal_loss(alpha=1), optimizer='adam', metrics=['accuracy', models.utils.f1_keras]) #binary_crossentropy
            elif loss == 'ce':
                model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy', models.utils.f1_keras]) 

            lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.7, \
                                                  patience=args.lr_schedule_round, verbose=1, mode='auto', min_lr=0.000001)
            config = {
                  'text_max_len': args.max_text_len,
                  'char_max_len': args.max_char_len,
                  'word_char_max_len': args.max_word_char_len,
                  'n_units': args.n_units,
                  'emb_dim': args.emb_dim,
                  'n_layers': args.n_layers,
                  'epochs': args.epochs,
                  "learning_rate": args.lr,
                  "model_name": model_name,
                  "loss": loss,
                  "use_features": use_features
                }

            if use_features == True:
                model_save_path = os.path.join(args.model_save_path, '{}_{}_with_features.h5'.format(model_name, config['loss']))
            else:
                model_save_path = os.path.join(args.model_save_path, '{}_{}_without_features.h5'.format(model_name, config['loss']))

            f1callback = models.utils.F1Callback(model, [word_val_inputs, char_val_inputs, subword_val_inputs, val_tfidf],\
                                      val_outputs, \
                                      filename=model_save_path, \
                                      patience=args.early_stopping_rounds)

            K.clear_session()

            if _has_wandb and args.wandb_logging:
                wandb.init(project='hate_speech_detection',config=config)
                model.fit([word_train_inputs, char_train_inputs, subword_train_inputs, train_tfidf], train_outputs, \
                      validation_data=([word_val_inputs, char_val_inputs, subword_val_inputs, val_tfidf], val_outputs), \
                          epochs=args.epochs,batch_size=args.train_batch_size, callbacks=[lr, f1callback, WandbCallback()], verbose=2)
            else:
                model.fit([word_train_inputs, char_train_inputs, subword_train_inputs, train_tfidf], train_outputs, \
                      validation_data=([word_val_inputs, char_val_inputs, subword_val_inputs, val_tfidf], val_outputs), \
                          epochs=args.epochs,batch_size=args.train_batch_size, callbacks=[lr, f1callback], verbose=1)


            model.load_weights(model_save_path)

            test_pred = model.predict([word_test_inputs, char_test_inputs, subword_test_inputs, test_tfidf])

            report = classification_report([idx2label[i] for i in test_outputs.argmax(-1)], \
                                           [idx2label[i] for i in test_pred.argmax(-1)])

            f1 = f1_score([idx2label[i] for i in test_outputs.argmax(-1)], \
                                           [idx2label[i] for i in test_pred.argmax(-1)], average='weighted')

            print (report, f1)
            
            results.loc[index,'config'] = str(config)
            results.loc[index, 'weighted_f1'] = f1_score([idx2label[i] for i in test_outputs.argmax(-1)], \
                                           [idx2label[i] for i in test_pred.argmax(-1)], average='weighted')
            results.loc[index, 'macro_f1'] = f1_score([idx2label[i] for i in test_outputs.argmax(-1)], \
                                           [idx2label[i] for i in test_pred.argmax(-1)], average='macro')
            
            index += 1

            results.to_csv(os.path.join(args.model_save_path,'results.csv'),index=False)

Running CS_ELMO_without_words without features for ce loss
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 50, 20)]     0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 50, 20, 128)  9088        input_3[0][0]                    
__________________________________________________________________________________________________
time_distributed (TimeDistribut (None, 50, 20, 128)  32896       embedding[0][0]                  
__________________________________________________________________________________________________
position_embedding (PositionEmb (None, 128)          2560        input_3[0][0]                    
___________________________________

W0110 13:09:20.258975 4591582720 network.py:957] Model was constructed with shape (None, 200) for input Tensor("input_4:0", shape=(None, 200), dtype=int32), but it was called on an input with incompatible shape (None, 18201).


Epoch 1/500


W0110 13:09:22.983972 4591582720 network.py:957] Model was constructed with shape (None, 200) for input Tensor("input_4:0", shape=(None, 200), dtype=int32), but it was called on an input with incompatible shape (None, 18201).


115/115 [==============================] - ETA: 0s - loss: 0.6656 - accuracy: 0.6266 - f1_keras: 0.3902

W0110 13:12:01.354864 4591582720 network.py:957] Model was constructed with shape (None, 200) for input Tensor("input_4:0", shape=(None, 200), dtype=int32), but it was called on an input with incompatible shape (None, 18201).
W0110 13:12:09.420538 4591582720 network.py:957] Model was constructed with shape (None, 200) for input Tensor("input_4:0", shape=(None, 200), dtype=int32), but it was called on an input with incompatible shape (None, 18201).



Score 0.41857506361323155. Model saved in ../models/hate_detection/CS_ELMO_without_words_ce_without_features.h5.
115/115 [==============================] - 168s 1s/step - loss: 0.6656 - accuracy: 0.6266 - f1_keras: 0.3902 - val_loss: 0.6008 - val_accuracy: 0.7199 - val_f1_keras: 0.4168 - lr: 0.0010
Epoch 2/500
115/115 [==============================] - ETA: 0s - loss: 0.6600 - accuracy: 0.6329 - f1_keras: 0.3869
Score 0.41857506361323155. Model not saved.
115/115 [==============================] - 182s 2s/step - loss: 0.6600 - accuracy: 0.6329 - f1_keras: 0.3869 - val_loss: 0.6052 - val_accuracy: 0.7199 - val_f1_keras: 0.4168 - lr: 0.0010
Epoch 3/500
115/115 [==============================] - ETA: 0s - loss: 0.6588 - accuracy: 0.6331 - f1_keras: 0.3922
Score 0.41857506361323155. Model not saved.
115/115 [==============================] - 163s 1s/step - loss: 0.6588 - accuracy: 0.6331 - f1_keras: 0.3922 - val_loss: 0.6110 - val_accuracy: 0.7199 - val_f1_keras: 0.4168 - lr: 0.0010
Epoch

Epoch 23/500
115/115 [==============================] - ETA: 0s - loss: 0.5594 - accuracy: 0.7168 - f1_keras: 0.6246
Score 0.7287833827893175. Model not saved.
115/115 [==============================] - 156s 1s/step - loss: 0.5594 - accuracy: 0.7168 - f1_keras: 0.6246 - val_loss: 0.5213 - val_accuracy: 0.7899 - val_f1_keras: 0.6718 - lr: 0.0010
Epoch 24/500
115/115 [==============================] - ETA: 0s - loss: 0.5629 - accuracy: 0.7160 - f1_keras: 0.6211
Score 0.7208468376535181. Model not saved.
115/115 [==============================] - 156s 1s/step - loss: 0.5629 - accuracy: 0.7160 - f1_keras: 0.6211 - val_loss: 0.5115 - val_accuracy: 0.8140 - val_f1_keras: 0.6698 - lr: 0.0010
Epoch 25/500
115/115 [==============================] - ETA: 0s - loss: 0.5578 - accuracy: 0.7143 - f1_keras: 0.6156
Score 0.7425701584716142. Model saved in ../models/hate_detection/CS_ELMO_without_words_ce_without_features.h5.
115/115 [==============================] - 156s 1s/step - loss: 0.5578 - accu

115/115 [==============================] - ETA: 0s - loss: 0.5860 - accuracy: 0.6985 - f1_keras: 0.5816
Score 0.6787293452908906. Model not saved.
115/115 [==============================] - 152s 1s/step - loss: 0.5860 - accuracy: 0.6985 - f1_keras: 0.5816 - val_loss: 0.5720 - val_accuracy: 0.7659 - val_f1_keras: 0.6351 - lr: 0.0010
Epoch 47/500
115/115 [==============================] - ETA: 0s - loss: 0.5928 - accuracy: 0.6949 - f1_keras: 0.5659
Score 0.6863725490196078. Model not saved.
115/115 [==============================] - 149s 1s/step - loss: 0.5928 - accuracy: 0.6949 - f1_keras: 0.5659 - val_loss: 0.5359 - val_accuracy: 0.7856 - val_f1_keras: 0.6359 - lr: 0.0010
Epoch 48/500
115/115 [==============================] - ETA: 0s - loss: 0.5802 - accuracy: 0.7045 - f1_keras: 0.5903
Score 0.6861641498346891. Model not saved.
115/115 [==============================] - 150s 1s/step - loss: 0.5802 - accuracy: 0.7045 - f1_keras: 0.5903 - val_loss: 0.5115 - val_accuracy: 0.7877 - val_f1

In [ ]:
results

In [ ]:
results.to_csv(os.path.join(args.model_save_path,'results.csv'),index=False)